In [ ]:
import sys
sys.path.append('src')

# Imports

In [ ]:
#export
import torch
import torch.nn as nn

# Code

In [48]:
class Test:
    def __init__(self, a,b,c):
        store_attr(self, locals())

In [ ]:
class Learner:
    def __init__(self, model, dls, loss_func, lr, cbs, opt_func):
        store_attr(self, locals())
        for cb in self.cbs: cb.learner = self

    def one_batch(self):
        self('before_batch')
        xb,yb = self.batch
        self.preds = self.model(xb)
        self.loss = self.loss_func(self.preds, yb)
        if self.model.training:
            self.loss.backward()
            self.opt.step()
        self('after_batch')

    def one_epoch(self, train):
        self.model.training = train
        self('before_epoch')
        dl = self.dls.train if train else self.dls.valid
        for self.num,self.batch in enumerate(progress_bar(dl, leave=False)):
            self.one_batch()
        self('after_epoch')
    
    def fit(self, n_epochs):
        self('before_fit')
        self.opt = self.opt_func(self.model.parameters(), self.lr)
        self.n_epochs = n_epochs
        try:
            for self.epoch in range(n_epochs):
                self.one_epoch(True)
                self.one_epoch(False)
        except CancelFitException: pass
        self('after_fit')
        
    def __call__(self, name):
        for cb in self.cbs: getattr(cb, name, None)()